In [1]:
import json
import os
import math
import random
import warnings
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm.notebook import tqdm
from scipy.optimize import least_squares

# Bibliotecas de Visualização
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

print("Bibliotecas carregadas com sucesso.")

Bibliotecas carregadas com sucesso.


In [2]:
# Configurações de Ambiente e Visualização
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Criação do diretório para imagens estáticas
OUTPUT_IMG_DIR = "imagens"
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

# Configurações Padrão (Fallback)
DEFAULT_CONFIG = {
    "hdop_threshold": 5.0,
    "bounds": {
        "min_lat": 51.185,
        "max_lat": 51.2361,
        "min_lon": 4.325,
        "max_lon": 4.459
    },

    "min_gateways_rssi": 3,
    "min_gateways_tdoa": 3,
    "min_gateways_ransac": 4,

    "rssi_ref": -21.7,
    "path_loss_n": 2.70,

    "ransac_iterations": 50,
    "ransac_consensus_threshold_m": 500,

    "kalman_q_val": 0.1,
    "kalman_r_cl": 500,
    "kalman_r_wcl": 300,
    "kalman_r_mlat_lse": 250,
    "kalman_r_tdoa_lm": 100,
    "kalman_r_tdoa_ransac": 50,
    "kalman_r_tdoa_bestgeom": 50,

    "kalman_warmup_steps": 10, 

    "total_area_km2": 53.07,

    "nn_config": {
        "gt_interval_seconds": 600,
        "hidden_neurons": 10,
        "epochs": 4000,
        "batch_size": 32,
        "test_split_size": 0.2,
        "validation_split_size": 0.1,
        "patience_early_stop": 50,
        "default_rssi": -120.0,
        "model_save_path": "modelo_nn_lora_gt.h5",
        "input_scaler_save_path": "input_scaler.joblib",
        "output_scaler_save_path": "output_scaler.joblib"
    },
    "use_calibrated_rssi_params": False, 
    "calibration_file_path": "gateway_calibration_results.json"
}

# Carregamento de Configurações
try:
    with open('config.json', 'r') as f:
        CONFIG = json.load(f)
    print("Configurações carregadas de 'config.json'.")
except FileNotFoundError:
    print("Aviso: 'config.json' não encontrado. Usando padrões.")
    CONFIG = DEFAULT_CONFIG

# Constantes Físicas
TOTAL_AREA_M2 = CONFIG.get('total_area_km2', 53.0) * 1e6
C_LIGHT = 299792458.0

# Configuração de Calibração RSSI
USE_CALIBRATED_RSSI = CONFIG.get('use_calibrated_rssi_params', False)
CALIBRATION_PARAMS = {"global_average": {}, "gateway_parameters": {}}

if USE_CALIBRATED_RSSI:
    calib_path = CONFIG.get('calibration_file_path', 'gateway_calibration_results.json')
    try:
        with open(calib_path, 'r') as f:
            CALIBRATION_PARAMS = json.load(f)
        
        if "global_average" not in CALIBRATION_PARAMS:
             CALIBRATION_PARAMS["global_average"] = {
                 'A_avg': CONFIG.get('rssi_ref', -21.7), 
                 'n_avg': CONFIG.get('path_loss_n', 2.70)
             }
        print(f"Modo CALIBRADO ativado usando: {calib_path}")
    except Exception as e:
        print(f"Erro ao carregar calibração ({e}). Revertendo para modo FIXO.")
        USE_CALIBRATED_RSSI = False

if not USE_CALIBRATED_RSSI:
    CALIBRATION_PARAMS["global_average"] = {
        'A_avg': CONFIG.get('rssi_ref', -21.7), 
        'n_avg': CONFIG.get('path_loss_n', 2.70)
    }
    print("Modo FIXO ativado.")

Configurações carregadas de 'config.json'.
Modo FIXO ativado.


In [3]:
def haversine(lat1, lon1, lat2, lon2):
    """Calcula a distância (metros) entre duas coordenadas (Haversine)."""
    if any(v is None for v in [lat1, lon1, lat2, lon2]):
        return float('inf')
        
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def latlon_to_xy(lat, lon, ref_lat, ref_lon):
    """Projeção local plana (equirretangular simplificada) em metros."""
    if any(v is None for v in [lat, lon]): return None, None
    x = haversine(ref_lat, ref_lon, ref_lat, lon) * np.sign(lon - ref_lon)
    y = haversine(ref_lat, ref_lon, lat, ref_lon) * np.sign(lat - ref_lat)
    return x, y

def xy_to_latlon(x, y, ref_lat, ref_lon):
    """Conversão inversa de XY metros para Lat/Lon."""
    R = 6371000
    new_lat = ref_lat + math.degrees(y / R)
    new_lon = ref_lon + math.degrees(x / (R * math.cos(math.radians(ref_lat))))
    return new_lat, new_lon

def is_within_bounds(lat, lon):
    """Verifica bounding box da área de interesse."""
    if lat is None or lon is None: return False
    b = CONFIG['bounds']
    return (b['min_lat'] <= lat <= b['max_lat']) and (b['min_lon'] <= lon <= b['max_lon'])

In [4]:
class KalmanFilter:
    """Filtro de Kalman Linear (Modelo Velocidade Constante - CV)."""
    def __init__(self, dt=1.0, R_val=10000, Q_val=0.1):
        self.F = np.eye(4) # Será atualizado com dt
        self.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
        self.Q = Q_val * np.eye(4)
        self.R = R_val * np.eye(2)
        self.P = 100 * np.eye(4)
        self.x = np.zeros((4, 1))
        self._update_dt(dt)

    def _update_dt(self, dt):
        self.F[0, 2] = dt
        self.F[1, 3] = dt
        self.F[2, 3] = 0 # CV model simplificado na matriz

    def predict(self, dt=None):
        if dt is not None: self._update_dt(dt)
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        if z is None: return
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(4) - K @ self.H) @ self.P


class ARFLFusion:
    """Adaptive Robust Fusion Layer (baseado em Fabris et al. 2025)."""
    def __init__(self, kf1, kf2):
        self.kf1 = kf1
        self.kf2 = kf2

    def step(self, z1_k, z2_k, dt):
        # 1. Predição Local
        self.kf1.predict(dt)
        self.kf2.predict(dt)
        x1_p, P1_p = self.kf1.x, self.kf1.P
        x2_p, P2_p = self.kf2.x, self.kf2.P

        # 2. Predição Fundida (Covariância Inversa)
        try:
            P1_inv, P2_inv = np.linalg.inv(P1_p), np.linalg.inv(P2_p)
            P_fused_p = np.linalg.inv(P1_inv + P2_inv)
            x_fused_p = P_fused_p @ (P1_inv @ x1_p + P2_inv @ x2_p)
        except np.linalg.LinAlgError:
            return x1_p, P1_p # Fallback simples

        # 3. Atualização Local Cruzada
        x1_u, P1_u = self._local_update(self.kf1, x_fused_p, P_fused_p, z1_k)
        x2_u, P2_u = self._local_update(self.kf2, x_fused_p, P_fused_p, z2_k)

        # 4. Estimativa Final Fundida
        if z1_k is None and z2_k is None:
            x_post, P_post = x_fused_p, P_fused_p
        elif z1_k is None:
            x_post, P_post = x2_u, P2_u
        elif z2_k is None:
            x_post, P_post = x1_u, P1_u
        else:
            P1_u_inv, P2_u_inv = np.linalg.inv(P1_u), np.linalg.inv(P2_u)
            P_post = np.linalg.inv(P1_u_inv + P2_u_inv)
            x_post = P_post @ (P1_u_inv @ x1_u + P2_u_inv @ x2_u)

        # 5. Feedback Loop (Atualiza filtros internos)
        self.kf1.x, self.kf1.P = x1_u, P1_u
        self.kf2.x, self.kf2.P = x2_u, P2_u

        return x_post, P_post

    def _local_update(self, kf, x_pred, P_pred, z):
        if z is None: return x_pred, P_pred
        H, R = kf.H, kf.R
        S = H @ P_pred @ H.T + R
        K = P_pred @ H.T @ np.linalg.inv(S)
        x_up = x_pred + K @ (z - H @ x_pred)
        P_up = (np.eye(4) - K @ H) @ P_pred
        return x_up, P_up

In [5]:
def rssi_to_distance(rssi, gw_id):
    """
    Converte RSSI em distância (metros).
    Usa parâmetros calibrados (A, n) se disponíveis, ou média global.
    """
    if USE_CALIBRATED_RSSI:
        # Busca params específicos ou usa fallback global
        params = CALIBRATION_PARAMS['gateway_parameters'].get(gw_id, CALIBRATION_PARAMS['global_average'])
        A = params.get('A_estimado', params.get('A_avg', CONFIG.get('rssi_ref', -58)))
        n = params.get('n_estimado', params.get('n_avg', CONFIG.get('path_loss_n', 2.2)))
    else:
        # Modo Fixo
        A = CALIBRATION_PARAMS['global_average']['A_avg']
        n = CALIBRATION_PARAMS['global_average']['n_avg']
    
    # Evita divisão por zero
    if n == 0: return float('inf')
    
    return 10**((A - rssi) / (10 * n))

def weighted_centroid_localization(rssi_data, locations):
    """
    Algoritmo WCL (Weighted Centroid Localization).
    Peso = 1 / (distância ^ g).
    """
    g = CONFIG.get('wcl_g_factor', 2.0)
    num_lat, num_lon, den = 0.0, 0.0, 0.0
    
    for gw_id, rssi in rssi_data.items():
        dist = rssi_to_distance(rssi, gw_id)
        if dist <= 0: continue
        
        weight = dist ** (-g)
        loc = locations[gw_id]
        
        num_lat += loc['latitude'] * weight
        num_lon += loc['longitude'] * weight
        den += weight
        
    if den > 0:
        return num_lat / den, num_lon / den
    return None, None

def multilateration_rssi_lse(distances, locations, ref_lat, ref_lon):
    """
    Multilateração baseada em distâncias RSSI usando Mínimos Quadrados Lineares (LSE).
    Linearização do sistema de equações de círculos.
    """
    ids = list(distances.keys())
    if len(ids) < 3: return None, None # Requer mínimo de 3 âncoras

    # Converte gateways para XY local
    gws_xy = {gid: latlon_to_xy(l['latitude'], l['longitude'], ref_lat, ref_lon) for gid, l in locations.items()}
    
    ref_id = ids[0]
    x1, y1 = gws_xy[ref_id]
    r1 = distances[ref_id]
    
    G, h = [], []
    for gid in ids[1:]:
        xi, yi = gws_xy[gid]
        ri = distances[gid]
        
        # Montagem da matriz Ax = b (Linearização)
        G.append([-2*(xi - x1), -2*(yi - y1)])
        h.append(ri**2 - r1**2 - (xi**2 + yi**2) + (x1**2 + y1**2))
        
    try:
        # Resolve o sistema linear
        pos = np.linalg.lstsq(np.array(G), np.array(h), rcond=None)[0]
        return pos[0], pos[1]
    except np.linalg.LinAlgError:
        return None, None

def solve_tdoa(sample_gws, gw_locs, ref_lat, ref_lon):
    """
    Solver TDoA usando Levenberg-Marquardt (Least Squares não-linear).
    Base: Minimizar a diferença entre (distância medida via tempo) e (distância geométrica).
    """
    # Garante ordenação por tempo para definir a referência (t0) corretamente
    sample_gws.sort(key=lambda g: g['rx_time_ns'])
    ref_gw = sample_gws[0]
    
    # Dados de entrada em XY
    gws_xy = [latlon_to_xy(gw_locs[g['id']]['latitude'], gw_locs[g['id']]['longitude'], ref_lat, ref_lon) for g in sample_gws]
    ref_pos = np.array(gws_xy[0])
    others_pos = gws_xy[1:]
    
    # Diferenças de distância medidas (convertidas de nanosegundos)
    # delta_d = c * (t_i - t_ref)
    delta_d_measured = [(g['rx_time_ns'] - ref_gw['rx_time_ns']) * 1e-9 * C_LIGHT for g in sample_gws[1:]]

    # Função de resíduo para o otimizador
    def residuals(est_pos, ref_p, other_ps, d_measures):
        # Distância estimada até a referência
        d_ref_est = np.linalg.norm(est_pos - ref_p)
        
        errors = []
        for op, d_meas in zip(other_ps, d_measures):
            # Distância estimada até o gateway i
            d_i_est = np.linalg.norm(est_pos - op)
            # Diferença geométrica estimada
            delta_d_est = d_i_est - d_ref_est
            # Erro = Modelo - Real
            errors.append(delta_d_est - d_meas)
        return errors

    # Chute inicial: média das posições dos gateways (centroide)
    initial_guess = np.mean(gws_xy, axis=0)
    
    try:
        result = least_squares(
            residuals, 
            initial_guess, 
            args=(ref_pos, others_pos, delta_d_measured), 
            method='lm'
        )
        return result.x[0], result.x[1]
    except ValueError:
        return None, None

def tdoa_ransac(gps_gws, gw_locs, ref_lat, ref_lon):
    """
    TDoA com RANSAC (Random Sample Consensus) para remoção de outliers.
    """
    # 1. Preparação
    if len(gps_gws) < CONFIG['min_gateways_ransac']: return None, None
    
    # Ordena globalmente para garantir referência consistente nos cálculos de erro
    gps_gws.sort(key=lambda g: g['rx_time_ns'])
    
    # Cache das posições XY para performance
    gws_xy_map = {
        g['id']: np.array(latlon_to_xy(gw_locs[g['id']]['latitude'], gw_locs[g['id']]['longitude'], ref_lat, ref_lon)) 
        for g in gps_gws
    }
    
    best_model_pos = None
    max_inliers = 0
    threshold = CONFIG['ransac_consensus_threshold_m']
    
    # 2. Loop RANSAC
    for _ in range(CONFIG['ransac_iterations']):
        try:
            # Amostra aleatória mínima (ex: 3 ou 4 gateways)
            sample = random.sample(gps_gws, CONFIG['min_gateways_tdoa'])
            
            # Gera modelo (posição candidata)
            model_x, model_y = solve_tdoa(sample, gw_locs, ref_lat, ref_lon)
            if model_x is None: continue
            
            model_pos = np.array([model_x, model_y])
            
            # 3. Passo de Consenso (Verificação de Inliers)
            current_inliers_count = 0
            
            # Distância do modelo até o gateway de referência (índice 0 da lista global ordenada)
            # Nota: TDoA requer uma referência comum para calcular os deltas
            ref_gw_global = gps_gws[0] 
            ref_pos_xy = gws_xy_map[ref_gw_global['id']]
            dist_ref_model = np.linalg.norm(model_pos - ref_pos_xy)
            
            for gw in gps_gws:
                # Pula o próprio gw de referência
                if gw['id'] == ref_gw_global['id']: 
                    current_inliers_count += 1
                    continue

                # a. Diferença de distância TEÓRICA (baseada no modelo)
                gw_pos_xy = gws_xy_map[gw['id']]
                dist_gw_model = np.linalg.norm(model_pos - gw_pos_xy)
                delta_d_model = dist_gw_model - dist_ref_model
                
                # b. Diferença de distância MEDIDA (baseada no tempo)
                delta_d_measured = (gw['rx_time_ns'] - ref_gw_global['rx_time_ns']) * 1e-9 * C_LIGHT
                
                # c. Verifica resíduo
                if abs(delta_d_model - delta_d_measured) < threshold:
                    current_inliers_count += 1
            
            # 4. Atualiza melhor modelo
            if current_inliers_count > max_inliers:
                max_inliers = current_inliers_count
                best_model_pos = (model_x, model_y)
                
        except (ValueError, np.linalg.LinAlgError):
            continue
            
    # Retorna a posição gerada pelo melhor consenso
    return best_model_pos

In [6]:
def load_dataset():
    try:
        with open('lorawan_antwerp_gateway_locations.json', 'r') as f:
            gw_locs = json.load(f)
        df = pd.read_json('lorawan_antwerp_2019_dataset.json')
    except FileNotFoundError as e:
        print(f"Erro fatal: {e.filename} não encontrado.")
        return None, None

    # Pré-processamento
    df = df[df['hdop'] <= CONFIG['hdop_threshold']].copy()
    
    # Extração segura de timestamp
    df['timestamp'] = pd.to_datetime([
        g[0]['rx_time']['time'] if g and 'rx_time' in g[0] else None 
        for g in df['gateways']
    ], utc=True).tz_convert(None)
    
    df.dropna(subset=['timestamp', 'dev_eui', 'latitude', 'longitude'], inplace=True)
    df.sort_values(['dev_eui', 'timestamp'], inplace=True)
    
    return df.groupby('dev_eui'), gw_locs

tracks_grp, gateway_locations = load_dataset()
if tracks_grp:
    print(f"Dataset pronto: {len(tracks_grp)} dispositivos identificados.")

Dataset pronto: 23 dispositivos identificados.


In [7]:
results_list = []
viz_list = []

if tracks_grp:
    for dev_id, track in tqdm(tracks_grp, desc="Simulando Trilhas"):
        if len(track) < CONFIG['kalman_warmup_steps']: continue

        # Inicialização de Filtros
        filters = {
            'WCL': KalmanFilter(R_val=CONFIG['kalman_r_wcl']**2),
            'MLAT-LSE': KalmanFilter(R_val=CONFIG['kalman_r_mlat_lse']**2),
            'TDoA-LM': KalmanFilter(R_val=CONFIG['kalman_r_tdoa_lm']**2),
            'TDoA-RANSAC': KalmanFilter(R_val=CONFIG['kalman_r_tdoa_ransac']**2)
        }
        arfl = ARFLFusion(
            KalmanFilter(R_val=CONFIG['kalman_r_wcl']**2), 
            KalmanFilter(R_val=CONFIG['kalman_r_tdoa_lm']**2)
        )

        # Referência Local
        ref_lat, ref_lon = track.iloc[0][['latitude', 'longitude']]
        last_ts_indep = None
        last_ts_arfl = None
        warmup = CONFIG['kalman_warmup_steps']

        for i, (idx, row) in enumerate(track.iterrows()):
            gt_x, gt_y = latlon_to_xy(row['latitude'], row['longitude'], ref_lat, ref_lon)
            
            # Delta T
            curr_ts = row['timestamp']
            dt = (curr_ts - last_ts_indep).total_seconds() if last_ts_indep else 1.0
            if dt <= 0: dt = 1.0
            last_ts_indep = curr_ts

            packet_res = {
                'dev_id': dev_id, 'ts': curr_ts, 
                'gt_lat': row['latitude'], 'gt_lon': row['longitude']
            }
            errors = {}
            
            # --- Medições ---
            # RSSI
            rssi_map = {gw['id']: gw['rssi'] for gw in row['gateways'] if gw['id'] in gateway_locations}
            meas_latlon, meas_xy = {}, {}
            
            if len(rssi_map) >= CONFIG['min_gateways_rssi']:
                wcl = weighted_centroid_localization(rssi_map, gateway_locations)
                if wcl[0]: meas_latlon['WCL'] = wcl
                
                dists = {k: rssi_to_distance(v, k) for k,v in rssi_map.items()}
                locs = {k: gateway_locations[k] for k in dists}
                lse = multilateration_rssi_lse(dists, locs, ref_lat, ref_lon)
                if lse[0]: meas_xy['MLAT-LSE'] = lse

            # TDoA
            gps_gws = [{**g, 'rx_time_ns': pd.to_datetime(g['rx_time']['time']).value} 
                       for g in row['gateways'] if g.get('rx_time', {}).get('ts_type') == 'GPS_RADIO']
            
            if len(gps_gws) >= CONFIG['min_gateways_tdoa']:
                lm = solve_tdoa(gps_gws, gateway_locations, ref_lat, ref_lon)
                if lm[0]: meas_xy['TDoA-LM'] = lm
            
            if len(gps_gws) >= CONFIG['min_gateways_ransac']:
                ransac = tdoa_ransac(gps_gws, gateway_locations, ref_lat, ref_lon)
                if ransac and ransac[0]: meas_xy['TDoA-RANSAC'] = ransac

            # --- Processamento Kalman Independente ---
            for method, kf in filters.items():
                kf.predict(dt)
                z = None
                
                # Tenta obter medição XY
                if method in meas_xy: 
                    z = np.array([[meas_xy[method][0]], [meas_xy[method][1]]])
                elif method in meas_latlon:
                    mx, my = latlon_to_xy(meas_latlon[method][0], meas_latlon[method][1], ref_lat, ref_lon)
                    if mx: z = np.array([[mx], [my]])
                
                kf.update(z)
                
                if i > warmup and is_within_bounds(*xy_to_latlon(kf.x[0,0], kf.x[1,0], ref_lat, ref_lon)):
                    err = math.sqrt((kf.x[0,0] - gt_x)**2 + (kf.x[1,0] - gt_y)**2)
                    errors[f"{method}+Kalman"] = err
                    packet_res[f"{method}_est_lat"], packet_res[f"{method}_est_lon"] = xy_to_latlon(kf.x[0,0], kf.x[1,0], ref_lat, ref_lon)

            # --- Fusão ARFL ---
            dt_arfl = (curr_ts - last_ts_arfl).total_seconds() if last_ts_arfl else 1.0
            if dt_arfl <= 0: dt_arfl = 1.0
            
            z_wcl = None
            if 'WCL' in meas_latlon:
                wx, wy = latlon_to_xy(*meas_latlon['WCL'], ref_lat, ref_lon)
                if wx: z_wcl = np.array([[wx], [wy]])
            
            z_tdoa = None
            if 'TDoA-LM' in meas_xy:
                z_tdoa = np.array([[meas_xy['TDoA-LM'][0]], [meas_xy['TDoA-LM'][1]]])

            # Executa fusão se houver pelo menos uma medição válida e novo timestamp
            if (z_wcl is not None or z_tdoa is not None):
                x_arfl, _ = arfl.step(z_wcl, z_tdoa, dt_arfl)
                last_ts_arfl = curr_ts # Atualiza timestamp apenas se houve step
                
                if i > warmup:
                    fx, fy = x_arfl[0,0], x_arfl[1,0]
                    if is_within_bounds(*xy_to_latlon(fx, fy, ref_lat, ref_lon)):
                        errors['ARFL'] = math.sqrt((fx - gt_x)**2 + (fy - gt_y)**2)
                        packet_res['ARFL_est_lat'], packet_res['ARFL_est_lon'] = xy_to_latlon(fx, fy, ref_lat, ref_lon)

            if errors:
                results_list.append(errors)
                # Adiciona erros ao pacote de visualização para facilitar cor/tamanho
                packet_res.update(errors) 
                viz_list.append(packet_res)

    df_results = pd.DataFrame(results_list)
    df_viz = pd.DataFrame(viz_list)
    print(f"Simulação Finalizada. Total de amostras: {len(df_results)}")

Simulando Trilhas:   0%|          | 0/23 [00:00<?, ?it/s]

Simulação Finalizada. Total de amostras: 127341


In [8]:
def save_plot(fig, filename):
    """Salva gráfico em alta resolução sem título (para relatórios) e exibe interativo."""
    # Remove título para salvar arquivo limpo
    layout_bkp = fig.layout.title.text
    fig.update_layout(title=None)
    filepath = os.path.join(OUTPUT_IMG_DIR, filename)
    fig.write_image(filepath, scale=2)
    
    # Restaura título e exibe
    fig.update_layout(title=layout_bkp)
    fig.show()

if not df_results.empty:
    # 1. Boxplot de Erros (Comparativo Global)
    fig_box = px.box(
        df_results, 
        points=False,
        title="Distribuição de Erro de Posicionamento por Método",
        labels={"variable": "Método", "value": "Erro (m)"}
    )
    fig_box.update_layout(template="plotly_white", showlegend=False)
    save_plot(fig_box, "boxplot_erros.png")

    # 2. CDF (Cumulative Distribution Function)
    # Preparar dados para CDF
    cdf_data = []
    methods = [c for c in df_results.columns if 'ARFL' in c or 'Kalman' in c]
    
    for method in methods:
        sorted_errors = np.sort(df_results[method].dropna())
        yvals = np.arange(len(sorted_errors)) / float(len(sorted_errors) - 1)
        cdf_data.append(pd.DataFrame({'Erro (m)': sorted_errors, 'CDF': yvals, 'Método': method}))
    
    if cdf_data:
        df_cdf = pd.concat(cdf_data)
        fig_cdf = px.line(
            df_cdf, x='Erro (m)', y='CDF', color='Método',
            title="CDF - Probabilidade Cumulativa de Erro",
            range_x=[0, 500] # Foca na região de interesse (0-500m)
        )
        fig_cdf.update_layout(template="plotly_white", yaxis_title="Probabilidade (P(X <= x))")
        save_plot(fig_cdf, "cdf_erros.png")

    # 3. Visualização de Trajetória (Amostra do melhor dispositivo)
    # Escolhe o dispositivo com mais pontos
    if not df_viz.empty:
        best_dev = df_viz['dev_id'].value_counts().idxmax()
        df_track = df_viz[df_viz['dev_id'] == best_dev].sort_values('ts')
        
        fig_map = go.Figure()
        
        # Ground Truth
        fig_map.add_trace(go.Scattermapbox(
            lat=df_track['gt_lat'], lon=df_track['gt_lon'],
            mode='lines+markers', name='Ground Truth',
            marker=dict(size=6, color='black'), line=dict(width=2)
        ))
        
        # ARFL
        if 'ARFL_est_lat' in df_track.columns:
            fig_map.add_trace(go.Scattermapbox(
                lat=df_track['ARFL_est_lat'], lon=df_track['ARFL_est_lon'],
                mode='lines+markers', name='ARFL Fusion',
                marker=dict(size=5, color='blue', opacity=0.7)
            ))
            
        # Configuração do Mapa (OpenStreetMap gratuito)
        fig_map.update_layout(
            mapbox_style="open-street-map",
            mapbox=dict(
                center=dict(lat=df_track['gt_lat'].mean(), lon=df_track['gt_lon'].mean()),
                zoom=13
            ),
            title=f"Trajetória: Dispositivo {best_dev}",
            margin={"r":0,"t":30,"l":0,"b":0}
        )
        save_plot(fig_map, "trajetoria_mapa.png")
        
    # Tabela Resumo Estatístico
    summary = df_results.describe().T[['mean', '50%', '75%', 'max']]
    summary.columns = ['Média', 'Mediana', '75%', 'Max']
    print("\n--- Resumo Estatístico dos Erros (Metros) ---")
    display(summary.sort_values('Média'))

else:
    print("Nenhum resultado gerado para visualização.")

RuntimeError: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome

